In [1]:
import numpy as np
import pandas as pd
import scanpy as sc
import seaborn as sns
import matplotlib.pyplot as plt
import decoupler as dc
from pydeseq2.dds import DeseqDataSet
from pydeseq2.ds import DeseqStats
import gseapy as gp
from gseapy.plot import barplot, dotplot

sns.set_context("paper", font_scale=2) # set some nice global plotting settings.

from IPython.display import Image # just for embedding plots we want you to try to make without providing code :)


<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


# Saving your processed adata

# Load processed adata

In [2]:
adata = sc.read("/data/class/cosmos_cluster6/PUBLIC/ref/example_cortex_adata_processed.h5ad")


In [3]:
adata.uns['log1p']["base"] = None # probably have to run this line, known scanpy bug.. https://github.com/scverse/scanpy/issues/2239#issuecomment-1104178881 


# Calculate cluster marker genes using scanpy

[Tutorial](https://scanpy.readthedocs.io/en/stable/tutorials/basics/clustering.html#differentially-expressed-genes-as-markers)

(You'll probably get a ton of PerformanceWarnings but they're fine to ignore, not errors just warnings about some internal scanpy code) 

In [4]:
# Obtain cluster-specific differentially expressed genes
sc.tl.rank_genes_groups(adata, groupby="leiden", method="t-test")


/opt/apps/python/3.8.0/lib/python3.8/site-packages/scanpy/tools/_rank_genes_groups.py:394: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  self.stats[group_name, 'names'] = self.var_names[global_indices]
/opt/apps/python/3.8.0/lib/python3.8/site-packages/scanpy/tools/_rank_genes_groups.py:396: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  self.stats[group_name, 'scores'] = scores[global_indices]
/opt/apps/python/3.8.0/lib/python3.8/site-packages/scanpy/tools/_rank_genes_groups.py:399: PerformanceWarning: DataFrame is highly fragme

## Dot plot of cluster markers

In [ ]:
sc.pl.rank_genes_groups_dotplot(
    adata, groupby="leiden", standard_scale="var", n_genes=5
)

/opt/apps/python/3.8.0/lib/python3.8/site-packages/scanpy/plotting/_dotplot.py:749: UserWarning: No data for colormapping provided via 'c'. Parameters 'cmap', 'norm' will be ignored
  dot_ax.scatter(x, y, **kwds)


## Example 1: looking further into cluster 0

### GO term analysis of cluster markers
Looking at so many genes is kind of overwhelming, sometimes GO terms help clarify what's going on!

In [ ]:
markers_cluster0 = sc.get.rank_genes_groups_df(adata, group="0")


In [ ]:
markers_cluster0.head()

In [ ]:
# gp.get_library_name(organism='Mouse') # available gene set libraries -- feel free to uncomment & explore others!!!


In [ ]:
markers_cluster0 = markers_cluster0[(markers_cluster0['pvals_adj'] < 0.01) & (markers_cluster0['logfoldchanges'] > 2)]
print(markers_cluster0.shape) # 100-1,000 typically a good number of genes for GO analysis

# Perform Reactome Pathway Enrichment Analysis
enr_cluster0 = gp.enrichr(gene_list=markers_cluster0['names'], 
                 organism='Mouse', # change organism to your organism
                 gene_sets=['Reactome_2022','GO_Biological_Process_2023'])

# Plot results
barplot(enr_cluster0.results, cutoff=0.05)
dotplot(enr_cluster0.results, cutoff=0.05)


#### Check specific genes from GO analysis
`enr.results` has more information not shown in plots such as the genes in your list that were in each GO term / reactome pathway.

In [ ]:
# can see which genes were in which reactome pathway 

enr_cluster0.results.head()



In [ ]:
# can also see which genes were in which GO term by filtering results

enr_cluster0.results[enr_cluster0.results['Gene_set'] == 'GO_Biological_Process_2023'].head()



In [ ]:
enr_cluster0.results['Genes'][0]

Heatmap-style UMAPs help show if marker expression is shared across clusters, and quickly tell you if expression is more global or specific to a small cluster

In [ ]:
plt.rcParams['figure.figsize'] = (10, 8)

In [ ]:
sc.pl.umap(adata, color=['Gria1','Camk2b'], size = 15)

In [ ]:
sc.pl.umap(adata, color=['leiden'], size = 15, legend_loc = "on data")


## Example 2: looking further into cluster 30

In [ ]:
# let's try a smaller cluster
markers_cluster30 = sc.get.rank_genes_groups_df(adata, group="30")


In [ ]:
markers_cluster30 = markers_cluster30[(markers_cluster30['pvals_adj'] < 0.01) & (markers_cluster30['logfoldchanges'] > 2)]
print(markers_cluster30.shape) # 100-1,000 typically a good number of genes for GO analysis

# Perform Reactome Pathway Enrichment Analysis
enr_cluster30 = gp.enrichr(gene_list=markers_cluster30['names'], 
                 organism='Mouse', # change organism to your organism
                 gene_sets=['Reactome_2022','GO_Biological_Process_2023'])

# Plot results
barplot(enr_cluster30.results, cutoff=0.05)
dotplot(enr_cluster30.results, cutoff=0.05)


In [ ]:
enr_cluster30.results[enr_cluster30.results['Gene_set'] == 'GO_Biological_Process_2023'].head()



In [ ]:
sc.pl.umap(adata, color=['Odad1','Dnai4','Spag17'], size = 15)

## Interpreting results

In our first example (cluster 0), some of the top-weighted GO terms had to do with neuronal systems, synaptic transmissing, etc. Since I know my data is in brain (left cerebral cortex), I can rule out the ones about heart and muscle as being super meaningful -- although of course these tissues are innervated, so those terms are still somewhat helpful! Overall, these terms tell me that the cluster 0 cell type is likely neuronal. 


Researching (Googling) marker genes is a tedious task but something that really helps in understanding your tissue and cell types! Also feel free to **ask TAs or Dr. Mortazavi** since we have spent a lot more time researching individual genes and might know something about at least one or two! :) 



In [ ]:
Image(filename='results/gria1.png')


These results from example #2 in cluster 30 are a little harder to interpret. However, Googling "cilium assembly brain cell type" had some interesting results regarding ependymal cells. In addition, other single-cell RNA-seq literature in brain also talks about some of the cluster-specific markers like Spag17. 

In [ ]:
Image(filename='results/cilia.png')


In [ ]:
Image(filename='results/spag17.png')


# Post-annotation pseudobulk differential expression analysis

## Create pseudobulk matrix
This code assumes your clusters are annotated with celltypes and your adata as a celltype obs column, e.g. `adata.obs['celltype']` exists!

In [ ]:
pdata = dc.get_pseudobulk(
    adata,
    sample_col='Sample',
    groups_col='celltype',
    obs=adata.obs,
    layer='raw_counts',
    mode='sum',
    min_cells=10,
    min_counts=1000
)

In [ ]:
pdata.obs.head()

## Compare two groups within each celltype
Save results to csv's. Takes a while to run.

In [ ]:
comparison = 'Genotype' # change to your comparison of interest!
group1 = 'CASTJ' # upregulated genes will be up in this group
group2 = 'B6J' # other group for comparison

In [25]:
types = adata.obs['celltype'].unique()

for this_type in types:
    print(this_type)
    gdata = pdata[pdata.obs[comparison].isin([group1, group2])].copy() # filter for the 2 groups of interest
    ctdata = gdata[gdata.obs['celltype'] == this_type].copy()

    # Count the number of samples in each group
    group_counts = ctdata.obs[comparison].value_counts()
    
    # Make sure each group is present
    if (group_counts.get(group1, 0) >= 1) and (group_counts.get(group2, 0) >= 1):
        dds = DeseqDataSet(adata=ctdata,
                           design_factors=comparison, 
                           refit_cooks=True)
        
        dds.deseq2()
        stat_res = DeseqStats(dds, contrast=[comparison, group1, group2])
        stat_res.summary()
        
        results = stat_res.results_df # get results df
        
        results['celltype'] = this_type
        results['nlog10padj'] = -np.log10(results['padj']) # make -log10(adj. p-value) column
        results.replace([np.inf, -np.inf], np.nan, inplace=True) # remove NAN and infinity values...
        results = results.dropna() # drop NAs
        results = results.reset_index().rename(columns={'index': 'gene_name'}) # set gene name to be its own column
        results.to_csv(f'results/deg_results_{this_type}_{group1}_vs_{group2}.csv', index=False)
    else:
        print(f"Skipping {this_type} because both groups are not present.")
        

Neuron
Fitting size factors...
... done in 0.00 seconds.

Fitting dispersions...
... done in 21.78 seconds.

Fitting dispersion trend curve...
... done in 26.32 seconds.

Fitting MAP dispersions...
... done in 15.67 seconds.

Fitting LFCs...
... done in 4.38 seconds.

Refitting 0 outliers.

Running Wald tests...
... done in 4.52 seconds.

Log2 fold change & Wald test p-value: Genotype CASTJ vs B6J


,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
gene_name,,,,,,
0610005C13Rik,3.210471,0.362502,0.738919,0.490584,0.623721,0.775332
0610006L08Rik,0.417554,0.967909,1.805823,0.535993,0.591963,NaN
0610009B22Rik,7.032265,-0.107681,0.540918,-0.199072,0.842207,0.917393
0610009E02Rik,45.031372,0.005866,0.149217,0.039310,0.968643,0.985624
0610009L18Rik,4.149971,0.138672,0.487919,0.284210,0.776249,0.878683
...,...,...,...,...,...,...
mt-Ti,0.226185,1.006925,3.816373,0.263843,0.791901,NaN
mt-Tl1,0.099974,-0.435760,3.872784,-0.112519,0.910412,NaN
mt-Tm,0.099974,-0.435760,3.872784,-0.112519,0.910412,NaN


/opt/apps/python/3.8.0/lib/python3.8/site-packages/pandas/core/arraylike.py:396: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)


Oligodendrocyte
Fitting size factors...
... done in 0.00 seconds.

Fitting dispersions...
... done in 11.45 seconds.

Fitting dispersion trend curve...
... done in 15.50 seconds.

Fitting MAP dispersions...
... done in 6.95 seconds.

Fitting LFCs...
... done in 2.50 seconds.

Refitting 0 outliers.

Running Wald tests...
... done in 4.50 seconds.

Log2 fold change & Wald test p-value: Genotype CASTJ vs B6J


,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
gene_name,,,,,,
0610005C13Rik,0.000000,NaN,NaN,NaN,NaN,NaN
0610006L08Rik,0.080965,0.116604,3.870039,0.030130,0.975964,NaN
0610009B22Rik,0.121242,1.559288,3.870118,0.402904,0.687019,NaN
0610009E02Rik,0.749481,-0.572073,1.633619,-0.350188,0.726198,NaN
0610009L18Rik,0.000000,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
mt-Ti,0.000000,NaN,NaN,NaN,NaN,NaN
mt-Tl1,0.000000,NaN,NaN,NaN,NaN,NaN
mt-Tm,0.000000,NaN,NaN,NaN,NaN,NaN


...

,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
gene_name,,,,,,
0610005C13Rik,0.000000,NaN,NaN,NaN,NaN,NaN
0610006L08Rik,0.000000,NaN,NaN,NaN,NaN,NaN
0610009B22Rik,0.483216,1.390946,2.229641,0.623843,0.532731,NaN
0610009E02Rik,0.000000,NaN,NaN,NaN,NaN,NaN
0610009L18Rik,0.234838,0.189070,3.051911,0.061951,0.950602,NaN
...,...,...,...,...,...,...
mt-Ti,0.000000,NaN,NaN,NaN,NaN,NaN
mt-Tl1,0.000000,NaN,NaN,NaN,NaN,NaN
mt-Tm,0.000000,NaN,NaN,NaN,NaN,NaN


VLMC
Fitting size factors...
... done in 0.00 seconds.

Fitting dispersions...
... done in 8.02 seconds.

Fitting dispersion trend curve...
... done in 11.33 seconds.

Fitting MAP dispersions...
... done in 3.78 seconds.

Fitting LFCs...
... done in 2.35 seconds.

Refitting 0 outliers.

Running Wald tests...
... done in 4.51 seconds.

Log2 fold change & Wald test p-value: Genotype CASTJ vs B6J


,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
gene_name,,,,,,
0610005C13Rik,0.000000,NaN,NaN,NaN,NaN,NaN
0610006L08Rik,0.000000,NaN,NaN,NaN,NaN,NaN
0610009B22Rik,0.000000,NaN,NaN,NaN,NaN,NaN
0610009E02Rik,0.133707,-0.694583,4.42567,-0.156944,0.875289,0.984037
0610009L18Rik,0.000000,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
mt-Ti,0.000000,NaN,NaN,NaN,NaN,NaN
mt-Tl1,0.000000,NaN,NaN,NaN,NaN,NaN
mt-Tm,0.000000,NaN,NaN,NaN,NaN,NaN


Pericyte
Skipping Pericyte because both groups are not present.
Epithelial
Skipping Epithelial because both groups are not present.
Fibroblast
Skipping Fibroblast because both groups are not present.
Ependymal
Skipping Ependymal because both groups are not present.


/opt/apps/python/3.8.0/lib/python3.8/site-packages/anndata/_core/anndata.py:121: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
/opt/apps/python/3.8.0/lib/python3.8/site-packages/anndata/_core/anndata.py:121: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
/opt/apps/python/3.8.0/lib/python3.8/site-packages/anndata/_core/anndata.py:121: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


## Dot plot of DEGs

In [ ]:
fname = "Oligodendrocyte_CASTJ_vs_B6J" # example!! change for your file!
results = pd.read_csv(f"results/deg_results_{fname}.csv")


In [ ]:
# at celltype level -- maybe not too interesting

In [ ]:
degs_up = results[(results['padj'] < 0.05) & (results['log2FoldChange'] > 2)] # increased lfc threshold for less genes to show
degs_down = results[(results['padj'] < 0.05) & (results['log2FoldChange'] < -2)] 



In [ ]:
sc.pl.dotplot(adata, degs_up['gene_name'].tolist(), 'celltype', 
              mean_only_expressed = False,
              dendrogram=True, log=True)



In [ ]:
sc.pl.dotplot(adata, degs_down['gene_name'].tolist(), 'celltype', 
              mean_only_expressed = False,
              dendrogram=True, log=True)



In [ ]:
# try filtering the adata for the celltype of interest and grouping by comparison


In [ ]:
adata_thistype = adata[adata.obs['celltype'] == 'Oligodendrocyte'].copy()

In [ ]:
sc.pl.dotplot(adata_thistype, degs_up['gene_name'].tolist(), 'Genotype', 
              mean_only_expressed = False,
              dendrogram=True, log=True)


In [ ]:
sc.pl.dotplot(adata_thistype, degs_down['gene_name'].tolist(), 'Genotype', 
              mean_only_expressed = False,
              dendrogram=True, log=True)


## Volcano plot

In [ ]:
! pip install adjustText # in case u dont have this package

from adjustText import adjust_text

In [ ]:
fname = "Oligodendrocyte_CASTJ_vs_B6J" # example!! change for your file!
results = pd.read_csv(f"results/deg_results_{fname}.csv")


In [ ]:
degs_up = results[(results['padj'] < 0.05) & (results['log2FoldChange'] > 1)]
degs_down = results[(results['padj'] < 0.05) & (results['log2FoldChange'] < -1)]


plt.figure(figsize=(7, 6))
sns.scatterplot(data=results, x='log2FoldChange', y='nlog10padj', alpha=0.5, 
                color='black', label = "Not significant")

# basically layer the subset dataframes on top of the original plot
sns.scatterplot(data=degs_up, x='log2FoldChange', y='nlog10padj', alpha=0.5, 
                color='red', label = f"Upregulated ({len(degs_up)} genes)")
sns.scatterplot(data=degs_down, x='log2FoldChange', y='nlog10padj', alpha=0.5, 
                color='blue', label = f"Downregulated ({len(degs_down)} genes)")

plt.xlabel('log2(Fold Change)')
plt.ylabel('-log10(adj. p-value)')
plt.title(fname)
plt.legend(bbox_to_anchor=(1, 1), loc='upper left') # move legend

# draw horizontal and vertical dashed lines
plt.axhline(1.3, color='black', linestyle='--')
plt.axvline(1, color='black', linestyle='--')
plt.axvline(-1, color='black', linestyle='--')

# sry this should help fix the overlaps
texts = []
for index, row in results.iterrows():
    if abs(row['log2FoldChange']) > 1 and row['nlog10padj'] > 10:
        texts.append(plt.text(row['log2FoldChange'], row['nlog10padj'], row['gene_name'], fontsize=10))

adjust_text(texts, arrowprops=dict(arrowstyle='-', color='black'))

plt.show()


## GO term enrichment of DEGs

Up- and down-regulated **separately**

In [ ]:
degs_up = results[(results['padj'] < 0.05) & (results['log2FoldChange'] > 0.2)] # not many DEGs in my case so had to decrease LFC threshold...

enr_up = gp.enrichr(gene_list=degs_up['gene_name'], 
                 organism='Mouse', # change organism to your organism
                 gene_sets=['Reactome_2022','GO_Biological_Process_2023'])

barplot(enr_up.results, cutoff=0.05)
dotplot(enr_up.results, cutoff=0.05)

In [ ]:
degs_down = results[(results['padj'] < 0.05) & (results['log2FoldChange'] < -1)]

enr_down = gp.enrichr(gene_list=degs_down['gene_name'], 
                 organism='Mouse', # change organism to your organism
                 gene_sets=['Reactome_2022','GO_Biological_Process_2023'])

barplot(enr_down.results, cutoff=0.05)
dotplot(enr_down.results, cutoff=0.05)